In [3]:
import warnings
warnings.filterwarnings("ignore")

In [4]:
from sklearn.metrics import (make_scorer, confusion_matrix, precision_score,
                             f1_score, roc_auc_score, accuracy_score,
                             recall_score)
from IPython.display import display, HTML
from sklearn.model_selection import StratifiedKFold, ParameterGrid

import sys
sys.path.append('/Users/samrelins/Documents/LIDA/ace_project/')
from data_prep.data_prep import *

# Initial Models: Analysis and Features

The below is a short(ish) script and summary of the data preparation and
modelling experiments I have performed. This is to be followed by a more in-depth analysis of a small selection of the best performing models - although, as we'll see, none of the models show impressive predictive accuracy at this point.

## Data Preparation Methods

The ACE dataset presents a couple of prominent considerations that need to be
 accounted for when preparing the data for modeling:

### 1. Categorical Encoding Methods:

Machine learning methods require categorical data to be represented
numerically for it to be interpretable. There are a number of ways to
approach
this, some of which are not possible in this setting because of the small amount of training data. I've focussed on two approaches:

**One Hot Encoding** - Each categorical feature is split into
individual categories and these categories are assigned a binary value, a
1 indicating the feature is present and 0 not present. For example, if we had
 the following data on the time of referral:

In [5]:
pd.DataFrame({
    "Referral Time": ["morning", "afternoon", "morning", "evening"],
}, index=[1,2,3,4])

,Referral Time
1,morning
2,afternoon
3,morning
4,evening


could be one-hot encoded as follows:

In [6]:
pd.DataFrame({
    "Referral Time Morning": [1, 0, 1, 0],
    "Referral Time Afternoon": [0, 1, 0, 0],
    "Referral Time Evening": [0, 0, 0, 1],
}, index=[1,2,3,4])

,Referral Time Morning,Referral Time Afternoon,Referral Time Evening
1,1,0,0
2,0,1,0
3,1,0,0
4,0,0,1


An issue with one-hot encoding is the creation of a large number of extra
features (one for each level of each categorical feature) i.e. the above took
 one category and made it into three. This crates a very "sparse" dataset
 (contains a lot of zeros that don't add much info) and can result in a very
 sparse model of the data i.e. a tree model that has to make hundreds
 of yes / no decisions on different binary categories before it can make a
 prediction. An alternative to this approach is to encode each category with
 a numerical representation of its value.

**Mean encoding / Feature encoding**


 Target encoding takes the target feature, in this case the need for hospital
  treatment, and encodes each categorical feature with the mean / proportion
  that applies to the individual "levels" of that category. Using the above example, we would calculate the proportion of referrals made in the morning / afternoon / evening that required hospital treatment, and use those proportions as numerical representations of the features. For example, if 15% of children referred in the morning required hospital treatment, and 5% and 18% for the kids referred in the afternoon and evening required hospital treatment, then the feature would look like this:

In [7]:
pd.DataFrame({
    "Referral Time": ["morning", "afternoon", "morning", "evening"],
    "Target Encoded Referral Time": [.15, .05, .18,
                                   .15],
}, index=[1,2,3,4])


,Referral Time,Target Encoded Referral Time
1,morning,0.15
2,afternoon,0.05
3,morning,0.18
4,evening,0.15


Note: One must be careful when using this approach, that "leakage" isn't
introduced
 into the dataset - that is, information about the target feature for that
 example being included in the explanatory variables for the same example.
 This can be avoided by ensuring that the target value for each example is
 left out when calculating its encodings.

Target encoding fixes the sparcity issue - each categorical feature remains
one feature rather than expanding, but it often results in overfitting - that
 is, when the model maps too closely to the examples it has seen in training
 and doesn't then generalise well when given new data.

### 2. Balancing Positive / Negative Examples:

The ACE dataset is heavily imbalanced i.e. only 16.5% of examples require
hospital treatment. Left as is, models can easily achieve high (83.5%)
accuracy by simply predicting ALL children can be treated by ACE. This
wouldn't be a very useful model!

To avoid this, efforts need to be made to balance the predictions made by
each model. Again, I have used two basic approaches to achieve this:

**1. Weighting Labels**:

The penalty a model is given for making an incorrect prediction can be
weighted to penalise the minority label incorrect guesses more
heavily. This discourages the model from simply guessing the majority label
over and over, as it gets a heavier penalty when it gets one of the minority
examples wrong.
 The
weight is usually chosen to be proportional to the imbalance i.e. if there
are 5 times more negative examples (children that can be treated by ACE) than
positive (children that require hospital treatment), then
 an incorrect
negative
guess is penalised 5 times more than an incorrect positive.

**2. SMOTE - Synthetic Minority Oversampling TEchnique**

This uses a statistical model to create synthetic examples from the minority
label to balance the number of positive / negative examples to 50/50. The
simplest form of oversampling is to simply duplicate the minority examples
over and over. SMOTE uses interpolation between the different minority
examples to create synthetic examples that roughly preserve the distribution of
 the
original examples.

### Data Preparation Pipeline

I've spent some time developing a "pipeline" or group of functions that can
automatically apply the above encoding and balancing techniques to the
data "at the flick of a switch". This means that, during training, the
different data preparation methods can be easily and consistently
applied to the data at runtime, without having to store many different versions
 of the ace dataset. The importance of this will become clear in the discussion of model
evaluation and cross validation below. Given the general utility of these
functions, and the fact they are fairly verbose, I have extracted them into a
 separate module - `data_prep.py` in which script and detailed documentation
 can be found.

A quick summary of the pipeline functions is as follows:

* `clean_data`: converts raw excel / csv data into a more python friendly format
* `fill_nas`: fills missing values in dataset with group means
* `add_features`: add the extra categorical features discussed in the data
analysis
* `return_train_test`: divides the dataset into consistent train and test
dataframes
* `add_synthetic_examples`: generates SMOTE (synthetic) examples and adds to
dataset
* `encode_and_scale`: applies various categorical encoding techniques and
min/max scales the data (for modelling techniques that require scaled data)

## Model Training and Evaluating Performance:

Having considered data preparation, we now need to define models to predict
the
hospital /
 community outcomes.

### Models and Parameters

The modelling techniques used are too numerous to attempt any discussion
here, but further exploration of the most successful techniques will be
included in
 the
more detailed discussion that will follow. Each modelling technique
includes a number of
parameters or "assumptions" that need to be specified when defining the model
 , and have a downstream effect on performance and prediction
 accuracy. To simplify and compartmentalise each of the models we wish to
 test along with its parameters (an extension of Ruaridh's work), we have
 a number of functions that return a model and a "parameter grid" of
  each of the parameters we wish to test.

Note: each function has a "balanced" argument allowing for the
calculation and use of balanced weights, which is not implemented in the
parameters of some models (hence the separate `scaled` keyword argument)

In [8]:
from sklearn.neighbors import KNeighborsClassifier

def return_knn_params(balanced=False):
    clf = KNeighborsClassifier()
    param_grid = {'n_neighbors': np.arange(1,10),
                    'weights': ['uniform','distance'],
                    'p': [1,2],
                    'metric':['minkowski','euclidean','manhattan'],
                    'n_jobs':[-2]}
    if balanced:
        print("no available balancing technique for nearest neighbours")
    return {"clf": clf, "param_grid": param_grid, "scaled": True}

In [9]:
from sklearn.svm import SVC
def return_svm_params(balanced=False):
    clf = SVC()
    param_grid = {'kernel': ['linear','rbf'],
                  'C': np.logspace(2,4,2), # np.logspace(2,5,6)
                  'gamma': np.logspace(-4,0.5,1)} # np.logspace(-4,0.5,10)}
    if balanced:
        param_grid["class_weight"] = "balanced",
    return {"clf": clf, "param_grid": param_grid, "scaled": True}

In [10]:
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF

def return_gaussian_process_params(balanced=False):
    clf = GaussianProcessClassifier(random_state=0, n_jobs=-2)
    kernels = [mul * RBF(length_scale)
                    for mul in np.arange(0.5, 2.5, 0.5)
                    for length_scale in np.arange(0.5, 2.5, 0.5)]
    param_grid = {'kernel': kernels,
                  'n_jobs': [-2]}
    if balanced:
        print("no available balancing technique for Gaussian Process")
    return {"clf": clf, "param_grid": param_grid, "scaled": True}

In [11]:
from sklearn.ensemble import RandomForestClassifier

def return_random_forest_params(balanced=False):
    clf = RandomForestClassifier(n_estimators=100)
    param_grid = {'max_depth': [4, 6, 10, 14, 20],
                  'n_estimators': [30, 100, 130, 300],
                  'min_samples_split': [2, 3, 10, 13, 30],
                  'max_features': [0.3, 0.4, 0.5, "auto"],
                  'n_jobs': [-2]}
    if balanced:
        param_grid["class_weight"] = "balanced",
    return {"clf": clf, "param_grid": param_grid, "scaled": False}

In [12]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.utils.class_weight import compute_class_weight

def return_grad_boost_params(balanced=False):
    clf = GradientBoostingClassifier(n_estimators=100,random_state=0)
    param_grid = {'learning_rate': [0.1, 0.05, 0.02, 0.01],
                  'n_estimators': [30, 100, 130, 300],
                  'max_depth': [4, 6, 10, 14, 20],
                  'min_samples_split': [3, 10, 13, 30],
                  'max_features': [x for x in np.linspace(0.2,0.4,4)]}
    return {"clf": clf, "param_grid": param_grid, "scaled": False,
            "weight_y": balanced}

In [13]:
from sklearn.ensemble import AdaBoostClassifier

def return_ada_boost_params(balanced=False):
    clf = AdaBoostClassifier(random_state=0)
    param_grid = {'n_estimators': [30, 100, 130, 300],
                  'learning_rate': [0.001,0.01,0.1,0.2,0.5]}
    return {"clf": clf, "param_grid": param_grid, "scaled": False,
            "weight_y":balanced}

In [14]:
from sklearn.naive_bayes import GaussianNB

def return_naive_bayes_params(balanced=False):
    clf = GaussianNB()
    param_grid = {'var_smoothing':  np.logspace(-11,-3,9,base=10)}
    return {"clf": clf, "param_grid": param_grid, "scaled": False,
            "weight_y":balanced}

In [15]:
from sklearn.linear_model import LogisticRegression

def return_lr_params(balanced=False):
    clf = LogisticRegression(random_state=0, max_iter=10000)
    param_grid = {'penalty' : ['l2'],
                  'solver': ["liblinear"],
                  'C' : np.logspace(-4, 4, 20)}
    if balanced:
        param_grid["class_weight"] = "balanced",
    return {"clf": clf, "param_grid": param_grid, "scaled": True}

In [16]:
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

def return_qda_params(balanced=False):
    clf = QuadraticDiscriminantAnalysis()
    param_grid = {'reg_param':  [0.0, 0.01, 0.03, 0.1, 0.3]}
    if balanced:
        print("no available balancing technique for QDA")
    return {"clf": clf, "param_grid": param_grid, "scaled": True}

In [17]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

def return_lda_params(balanced=False):
    clf = LinearDiscriminantAnalysis()
    param_grid = {'solver':  ["svd", "lsqr", "eigen"],
                  "shrinkage": [None, "auto", 0.1, 0.3, 0.8, 1]}
    if balanced:
        print("no available balancing technique for LDA")
    return {"clf": clf, "param_grid": param_grid, "scaled": True}


### Cross Validation and Grid Search

For those keeping score, we now have several different data preparation techniques,
models. We can't use them all simultaneously (well - technically that is
actually possible but would be a needlessly complex solution) and so we need
to decide on the best combination. To evaluate each possible permutation of
data preparation / model / parameters we
can use a technique called "cross validation": dividing the
training data into k
groups, training the model on k-1 of these groups leaving one group aside, and
 evaluating the model's predictions against the group held aside. Doing this
 ensures the model is never evaluated on examples it has already seen. This
 technique is used in conjunction with a parameter optimisation method called
  "grid search" - this iterates through each possible combination of the
  specified parameters, scoring each individually. The best combination of
  parameters can then be established.

There are several "out of the box" implementations of these methods that can
be applied in most use cases. However, these functions require you to specify
 a pipeline that can be compartmentalised into distinct stages and applied
 across the whole training dataset. This isn't practical in this case as:

 * The synthesizing and encoding stages can't be divorced from one another,
 otherwise the SMOTE examples may extrapolate between target-encoded
 features to produce nonsense examples
 * Only the training splits can include synthetic data - otherwise model
 performance will be based in part on its ability to predict synthetic data
 and will result in a biased cross validation score

Therefore, I've spent some time developing a custom cross validation and grid
 search loop that produces unbiased validation scores. The function includes
 the different data preparation techniques and pipeline functions discussed
 above:

In [18]:
import warnings
warnings.filterwarnings("ignore")
from sklearn.metrics import (make_scorer, confusion_matrix, precision_score,
                             f1_score, roc_auc_score, accuracy_score,
                             recall_score)
from sklearn.model_selection import StratifiedKFold, ParameterGrid

import sys
sys.path.append('/Users/samrelins/Documents/LIDA/ace_project/')
from data_prep.data_prep import *

# custom scoring functions for CV loop
true_neg = make_scorer(lambda y, y_pred: confusion_matrix(y, y_pred)[0][0])
false_neg = make_scorer(lambda y, y_pred: confusion_matrix(y, y_pred)[1][0])
true_pos = make_scorer(lambda y, y_pred: confusion_matrix(y, y_pred)[1][1])
false_pos = make_scorer(lambda y, y_pred: confusion_matrix(y, y_pred)[0][1])
precision = make_scorer(precision_score, zero_division=0)

# dict of scoring functions
SCORING = {
    "f1": make_scorer(f1_score),
    "roc_auc": make_scorer(roc_auc_score),
    "accuracy": make_scorer(accuracy_score),
    "recall": make_scorer(recall_score),
    "precision": make_scorer(precision_score),
    "true_pos": true_pos,
    "true_neg": true_neg,
    "false_pos": false_pos,
    "false_neg": false_neg
}


def score_classifier(clf, X, y):
    """
    Scores a classifier against metrics in SCORING dict

    :param clf: (object: sklearn classifier) classifier to be scored
    :param X: (object: pandas DataFrame) matrix of training vectors
    :param y: (object: pandas Series) vector of target labels
    :return: (dict) group of {score function name: score} pairs
    """

    scores = {}
    for name, scorer in SCORING.items():
        scores[name] = scorer(clf, X, y)
    return scores


def cv_score_classifier(clf, X_train, y_train, params,
                        cat_encoder="one_hot",
                        add_synthetic=False,
                        scaled=False,
                        n_splits=3,
                        weight_y=False):
    """
    Custom CV loop to score classifier functions

    Implemented to account for SMOTE example generation and target encoding -
    both should only be performed on training data and not validation data -
    not possible to achieve this separation using the sklearn pipeline and
    GridSearchCV.

    :param clf: (object: sklearn classifier) classifier to train and score
    :param X_train: (object: pandas DataFrame) Explanatory Training data
    :param y_train: (object: pandas Series) Training data labels
    :param params: (dict) parameters for classifier
    :param cat_encoder: (str: "one_hot") categorical encoder for data
    either "one_hot" / "target"
    :param add_synthetic: (bool: False) set True to add SMOTE examples before
    training
    :param scaled: (bool: False) set True to scale numeric features
    :param n_splits: (int: 3) number of splilts for CV loop
    :param weight_y: (bool: False) set True if clf requires sample_weights
    :return:
    """

    # create splits for CV loop
    splitter = StratifiedKFold(n_splits=n_splits, random_state=1)
    splits = list(splitter.split(X_train, y_train))

    total_cv_scores = {} # dict to store cumulative CV scores
    for train_idxs, val_idxs  in splits:
        # divide data into train and validation sets for this cv loop
        cv_X_train, cv_y_train, X_val, y_val = (X_train.iloc[train_idxs],
                                                y_train.iloc[train_idxs],
                                                X_train.iloc[val_idxs],
                                                y_train.iloc[val_idxs])

        if add_synthetic: # add SMOTE examples to balance data if required
            cv_X_train, cv_y_train = add_synthetic_examples(cv_X_train, cv_y_train)

        # encode categorical features and scale numeric if required
        cv_X_train, X_val, = encode_and_scale(
            cv_X_train, cv_y_train, X_val,
            cat_encoder=cat_encoder,
            scaled=scaled)

        if weight_y:
            # calculate array of weights for y labels
            pos_weight, neg_weight = compute_class_weight(
                class_weight="balanced",
                classes=[1,0],
                y=cv_y_train)
            y_weights = cv_y_train.apply(lambda y: pos_weight if y else neg_weight)
            # train model using parameters, weights and cv loop data
            cv_clf = (clf
                      .set_params(**params)
                      .fit(cv_X_train, cv_y_train, sample_weight=y_weights))
        else:
            # train model using parameters and cv loop data
            cv_clf = (clf
                      .set_params(**params)
                      .fit(cv_X_train, cv_y_train))

        # score classifier on cv validation set and add scores to total
        scores = score_classifier(cv_clf, X_val, y_val)
        if total_cv_scores:
            for key, value in scores.items():
                total_cv_scores[key] += value
        else:
            total_cv_scores = scores

    mean_cv_scores = {}
    for key, value in total_cv_scores.items():
        mean_cv_scores[key] = value / n_splits

    return mean_cv_scores


def param_search_classifier(param_grid, **kwargs):
    """
    custom param grid search to compliment cv_score_classifier function

    :param param_grid: (dict) parameters on which to perform grid search
    :param kwargs: arguments for cv_score_classifier function
    :return: (dict: best_scores, dict: best_params) scores and parameters for
    highest scoring model
    """
    param_grid = ParameterGrid(param_grid)
    # variable to store best param combo and relevant scores
    best_scores = {}
    best_params = {}
    for params in param_grid:
        mean_cv_scores = cv_score_classifier(params=params,
                                             **kwargs)
        if not best_scores:
            best_scores = mean_cv_scores
            best_params = params
        elif mean_cv_scores["f1"] > best_scores["f1"]:
            best_scores = mean_cv_scores
            best_params = params

    return best_scores, best_params

## Performance Metrics

The cross validation loop outputs the following metrics, used to measure
model performance:

* **True Positive / False Positive / True Negative / False Negative**: Fairly
self
 explanatory. A true positive in this context is an example a model correctly
  states requires hospital treatment, a true negative is an example the model
   states needs hospital treatment when it doesn't, and so on....
* **Accuracy**: Again fairly self explanatory. The proportion of
 correct predictions
* **Precision**: the proportion of positive guesses that are
correct i.e. if a model has a precision of 75%, 3 out of every 4 times it
predicts that hospital treatment is needed it is correct.
* **Recall**: the proportion of positive examples in the dataset that the
model correctly predicts i.e. if there are 50 examples requiring hospital
treatment and the model correctly identifies 40 of them, it has an 80% recall.
* **ROC/AUC**: this is a measure of the tradeoff between precision and
recall, but is a little complex to define here. A 0.5 ROC/AUC is
representative of random chance and 1 is a perfect model.
* **F1 Score**: the f1 score is another measure of the tradeoff between precision and recall. It is a weighted average of the two and ranges from 0 (worst) to 1 (perfect)

## Tests

The following is the (perhaps long awaited!) output from the training /
validation. Scores are broken down by data preparation methods and then model
 type - the best performing model for each is selected from the cv loop and
 displayed in the results:

In [20]:
techniques_dict = {'K Nearest Neighbours': return_knn_params,
                   'Support Vector Machines': return_svm_params,
                   'Gaussian Process': return_gaussian_process_params,
                   'Random Forest Classifier': return_random_forest_params,
                   'Gradient Boosting Classifier': return_grad_boost_params,
                   'Ada Boost classifier': return_ada_boost_params,
                   'Gaussian Naieve Bayes': return_naive_bayes_params,
                   'Logistic Regression': return_lr_params,
                   'Quadratic Discriminant Analysis': return_qda_params}

# data_prep_types = ["one_hot_balanced", "target_balanced",
#                     "one_hot_resampled", "target_resampled"]
data_prep_types = ["one_hot_balanced"]

data_loc = "/Users/samrelins/Documents/LIDA/ace_project/data/ace_data_orig.csv"
ace_data_orig = pd.read_csv(data_loc)
X_train, y_train, X_test, y_test = return_train_test(ace_data_orig)

best_params = {}
best_model_scores = {}

for data_prep_type in data_prep_types:
    ### uncomment this and other print statements for output of loop progress
    print(50* '=')
    print(f"Data Prep: {data_prep_type}")
    print(50* '=')

    cat_encoder = "one_hot" if "one_hot" in data_prep_type else "target"
    balanced = True if "balanced" in data_prep_type else False
    add_synthetic = True if "resampled" in data_prep_type else False

    scores_list = []
    best_loop_params = {}
    for model_name, model_params_f in techniques_dict.items():
        print(f"fitting {model_name}......")
        model_best_scores, model_best_params = param_search_classifier(
            **model_params_f(balanced=balanced),
            X_train=X_train,
            y_train=y_train,
            cat_encoder=cat_encoder,
            add_synthetic=add_synthetic)
        scores_list.append(model_best_scores)
        best_loop_params[model_name] = model_best_params
        print("done.")

    best_model_scores[data_prep_type] = pd.DataFrame(scores_list, 
                                                     index=techniques_dict.keys())
    best_params[data_prep_type] = best_loop_params

Data Prep: one_hot_balanced
fitting K Nearest Neighbours......
no available balancing technique for nearest neighbours
done.
fitting Support Vector Machines......
done.
fitting Gaussian Process......
no available balancing technique for Gaussian Process
done.
fitting Random Forest Classifier......
done.
fitting Gradient Boosting Classifier......
done.
fitting Ada Boost classifier......
done.
fitting Gaussian Naieve Bayes......
done.
fitting Logistic Regression......
done.
fitting Quadratic Discriminant Analysis......
no available balancing technique for QDA
done.


In [21]:
fit_features = []
rename_features = {}
for feature in SCORING.keys():
    feature_name = "mean_test_" + feature
    fit_features.append(feature_name)
    rename_features[feature_name] = feature

fit_features = [fit_features[0]] + ["std_test_f1"] + fit_features[1:]
rename_features["mean_test_f1"] = "f1 Mean"
rename_features["std_test_f1"] = "f1 Std"

def highlight_good_scores_green(df):
    good_accuracy = df["accuracy"] > 0.6
    good_recall = df["recall"] > 0.4
    good_precision = df["precision"] > 0.2
    highlight = good_recall & good_accuracy & good_precision
    if highlight:
        return [f"background-color: green"] * 9
    else:
        return [f"background-color:"] * 9


for name, df in best_model_scores.items():
    display(HTML(f"<h2>{name.replace('_', ' ').title()}"))
    display(df.rename(rename_features, axis=1)
            .style.apply(highlight_good_scores_green, axis=1))


,f1,roc_auc,accuracy,recall,precision,true_pos,true_neg,false_pos,false_neg
K Nearest Neighbours,0.166724,0.496545,0.718031,0.162768,0.186370,3.000000,75.000000,15.333333,15.333333
Support Vector Machines,0.283532,0.545859,0.582909,0.490253,0.199466,9.000000,54.333333,36.000000,9.333333
Gaussian Process,0.000000,0.500000,0.831295,0.000000,0.000000,0.000000,90.333333,0.000000,18.333333
Random Forest Classifier,0.280072,0.561015,0.739382,0.291423,0.279693,5.333333,75.000000,15.333333,13.000000
Gradient Boosting Classifier,0.306978,0.576768,0.693397,0.400585,0.251190,7.333333,68.000000,22.333333,11.000000
Ada Boost classifier,0.291243,0.558274,0.629148,0.452242,0.222510,8.333333,60.000000,30.333333,10.000000
Gaussian Naieve Bayes,0.274618,0.484982,0.230066,0.870370,0.163084,16.000000,9.000000,81.333333,2.333333
Logistic Regression,0.296357,0.560654,0.607402,0.490253,0.212440,9.000000,57.000000,33.333333,9.333333
Quadratic Discriminant Analysis,0.031746,0.497722,0.815947,0.017544,0.166667,0.333333,88.333333,2.000000,18.000000


In [27]:
techniques_dict = {'K Nearest Neighbours': return_knn_params,
                   'Support Vector Machines': return_svm_params,
                   'Gaussian Process': return_gaussian_process_params,
                   'Random Forest Classifier': return_random_forest_params,
                   'Gradient Boosting Classifier': return_grad_boost_params,
                   'Ada Boost classifier': return_ada_boost_params,
                   'Gaussian Naieve Bayes': return_naive_bayes_params,
                   'Logistic Regression': return_lr_params,
                   'Quadratic Discriminant Analysis': return_qda_params}

data_prep_types = ["one_hot_balanced", "target_balanced",
                    "one_hot_resampled", "target_resampled"]

data_loc = "/Users/samrelins/Documents/LIDA/ace_project/data/ace_data_orig.csv"
ace_data_orig = pd.read_csv(data_loc)
X_train, y_train, X_test, y_test = return_train_test(ace_data_orig)

best_params = {}
best_model_scores = {}

for data_prep_type in data_prep_types:
    ### uncomment this and other print statements for output of loop progress
    print(50* '=')
    print(f"Data Prep: {data_prep_type}")
    print(50* '=')

    cat_encoder = "one_hot" if "one_hot" in data_prep_type else "target"
    balanced = True if "balanced" in data_prep_type else False
    add_synthetic = True if "resampled" in data_prep_type else False

    scores_list = []
    best_loop_params = {}
    for model_name, model_params_f in techniques_dict.items():
        print(f"fitting {model_name}......")
        model_best_scores, model_best_params = param_search_classifier(
            **model_params_f(balanced=balanced),
            X_train=X_train,
            y_train=y_train,
            cat_encoder=cat_encoder,
            add_synthetic=add_synthetic)
        scores_list.append(model_best_scores)
        best_loop_params[model_name] = model_best_params
        print("done.")

    best_model_scores[data_prep_type] = pd.DataFrame(scores_list,
                                                     index=techniques_dict.keys())
    best_params[data_prep_type] = best_loop_params

Data Prep: one_hot_balanced
fitting K Nearest Neighbours......
no available balancing technique for nearest neighbours
done.
fitting Support Vector Machines......
done.
fitting Gaussian Process......
no available balancing technique for Gaussian Process
done.
fitting Random Forest Classifier......
done.
fitting Gradient Boosting Classifier......
done.
fitting Ada Boost classifier......
done.
fitting Gaussian Naieve Bayes......
done.
fitting Logistic Regression......
done.
fitting Quadratic Discriminant Analysis......
no available balancing technique for QDA
done.
Data Prep: target_balanced
fitting K Nearest Neighbours......
no available balancing technique for nearest neighbours
done.
fitting Support Vector Machines......
done.
fitting Gaussian Process......
no available balancing technique for Gaussian Process
done.
fitting Random Forest Classifier......
done.
fitting Gradient Boosting Classifier......
done.
fitting Ada Boost classifier......
done.
fitting Gaussian Naieve Bayes......


In [28]:
fit_features = []
rename_features = {}
for feature in SCORING.keys():
    feature_name = "mean_test_" + feature
    fit_features.append(feature_name)
    rename_features[feature_name] = feature

fit_features = [fit_features[0]] + ["std_test_f1"] + fit_features[1:]
rename_features["mean_test_f1"] = "f1 Mean"
rename_features["std_test_f1"] = "f1 Std"

def highlight_good_scores_green(df):
    good_accuracy = df["accuracy"] > 0.6
    good_recall = df["recall"] > 0.4
    good_precision = df["precision"] > 0.2
    highlight = good_recall & good_accuracy & good_precision
    if highlight:
        return [f"background-color: green"] * 9
    else:
        return [f"background-color:"] * 9


for name, df in best_model_scores.items():
    display(HTML(f"<h2>{name.replace('_', ' ').title()}"))
    display(df.rename(rename_features, axis=1)
            .style.apply(highlight_good_scores_green, axis=1))


,f1,roc_auc,accuracy,recall,precision,true_pos,true_neg,false_pos,false_neg
K Nearest Neighbours,0.166724,0.496545,0.718031,0.162768,0.186370,3.000000,75.000000,15.333333,15.333333
Support Vector Machines,0.154184,0.501999,0.739297,0.144250,0.166382,2.666667,77.666667,12.666667,15.666667
Gaussian Process,0.000000,0.500000,0.831295,0.000000,0.000000,0.000000,90.333333,0.000000,18.333333
Random Forest Classifier,0.184048,0.521553,0.757759,0.164717,0.216640,3.000000,79.333333,11.000000,15.333333
Gradient Boosting Classifier,0.281185,0.561076,0.714888,0.328460,0.253256,6.000000,71.666667,18.666667,12.333333
Ada Boost classifier,0.288752,0.543789,0.534347,0.559454,0.208484,10.333333,47.666667,42.666667,8.000000
Gaussian Naieve Bayes,0.274618,0.484982,0.230066,0.870370,0.163084,16.000000,9.000000,81.333333,2.333333
Logistic Regression,0.296357,0.560654,0.607402,0.490253,0.212440,9.000000,57.000000,33.333333,9.333333
Quadratic Discriminant Analysis,0.031746,0.497722,0.815947,0.017544,0.166667,0.333333,88.333333,2.000000,18.000000


,f1,roc_auc,accuracy,recall,precision,true_pos,true_neg,false_pos,false_neg
K Nearest Neighbours,0.167537,0.511745,0.754729,0.145224,0.199252,2.666667,79.333333,11.000000,15.666667
Support Vector Machines,0.185827,0.506840,0.721061,0.183236,0.191626,3.333333,75.000000,15.333333,15.000000
Gaussian Process,0.000000,0.500000,0.831295,0.000000,0.000000,0.000000,90.333333,0.000000,18.333333
Random Forest Classifier,0.174976,0.514105,0.745328,0.164717,0.195833,3.000000,78.000000,12.333333,15.333333
Gradient Boosting Classifier,0.247101,0.541846,0.720806,0.271930,0.227020,5.000000,73.333333,17.000000,13.333333
Ada Boost classifier,0.215516,0.456557,0.429296,0.496101,0.138847,9.000000,37.666667,52.666667,9.333333
Gaussian Naieve Bayes,0.255666,0.552819,0.693199,0.342105,0.209528,6.333333,69.000000,21.333333,12.000000
Logistic Regression,0.288277,0.502827,0.196228,0.964912,0.169629,17.666667,3.666667,86.666667,0.666667
Quadratic Discriminant Analysis,0.068376,0.463024,0.671027,0.148148,0.044444,2.666667,70.333333,20.000000,15.666667


,f1,roc_auc,accuracy,recall,precision,true_pos,true_neg,false_pos,false_neg
K Nearest Neighbours,0.274338,0.534789,0.564560,0.490253,0.190524,9.000000,52.333333,38.000000,9.333333
Support Vector Machines,0.137856,0.473425,0.690197,0.146199,0.130554,2.666667,72.333333,18.000000,15.666667
Gaussian Process,0.150014,0.493775,0.723893,0.146199,0.155806,2.666667,76.000000,14.333333,15.666667
Random Forest Classifier,0.257058,0.540766,0.681051,0.328460,0.215744,6.000000,68.000000,22.333333,12.333333
Gradient Boosting Classifier,0.206999,0.522511,0.736239,0.199805,0.219124,3.666667,76.333333,14.000000,14.666667
Ada Boost classifier,0.267751,0.513769,0.519057,0.506823,0.195378,9.333333,47.000000,43.333333,9.000000
Gaussian Naieve Bayes,0.268693,0.506829,0.432439,0.618908,0.171726,11.333333,35.666667,54.666667,7.000000
Logistic Regression,0.298059,0.530238,0.291284,0.890838,0.179028,16.333333,15.333333,75.000000,2.000000
Quadratic Discriminant Analysis,0.252112,0.517671,0.582824,0.419103,0.180556,7.666667,55.666667,34.666667,10.666667


,f1,roc_auc,accuracy,recall,precision,true_pos,true_neg,false_pos,false_neg
K Nearest Neighbours,0.235951,0.501573,0.579907,0.383041,0.170900,7.000000,56.000000,34.333333,11.333333
Support Vector Machines,0.168868,0.492066,0.696455,0.183236,0.158689,3.333333,72.333333,18.000000,15.000000
Gaussian Process,0.164396,0.493816,0.711802,0.164717,0.165079,3.000000,74.333333,16.000000,15.333333
Random Forest Classifier,0.192026,0.495891,0.653415,0.257310,0.155168,4.666667,66.333333,24.000000,13.666667
Gradient Boosting Classifier,0.177200,0.502628,0.714719,0.182261,0.175613,3.333333,74.333333,16.000000,15.000000
Ada Boost classifier,0.247113,0.478584,0.423378,0.562378,0.158421,10.333333,35.666667,54.666667,8.000000
Gaussian Naieve Bayes,0.231782,0.486864,0.533299,0.417154,0.164840,7.666667,50.333333,40.000000,10.666667
Logistic Regression,0.293175,0.511598,0.199286,0.982456,0.172457,18.000000,3.666667,86.666667,0.333333
Quadratic Discriminant Analysis,0.236361,0.508722,0.546466,0.452242,0.168651,8.333333,51.000000,39.333333,10.000000


In [18]:
fit_features = []
rename_features = {}
for feature in SCORING.keys():
    feature_name = "mean_test_" + feature
    fit_features.append(feature_name)
    rename_features[feature_name] = feature

fit_features = [fit_features[0]] + ["std_test_f1"] + fit_features[1:]
rename_features["mean_test_f1"] = "f1 Mean"
rename_features["std_test_f1"] = "f1 Std"

def highlight_good_scores_green(df):
    good_accuracy = df["accuracy"] > 0.6
    good_recall = df["recall"] > 0.4
    good_precision = df["precision"] > 0.2
    highlight = good_recall & good_accuracy & good_precision
    if highlight:
        return [f"background-color: green"] * 9
    else:
        return [f"background-color:"] * 9


for name, df in best_model_scores.items():
    display(HTML(f"<h2>{name.replace('_', ' ').title()}"))
    display(df.rename(rename_features, axis=1)
            .style.apply(highlight_good_scores_green, axis=1))


,f1,roc_auc,accuracy,recall,precision,true_pos,true_neg,false_pos,false_neg
K Nearest Neighbours,0.166724,0.496545,0.718031,0.162768,0.186370,3.000000,75.000000,15.333333,15.333333
Support Vector Machines,0.283532,0.545859,0.582909,0.490253,0.199466,9.000000,54.333333,36.000000,9.333333
Gaussian Process,0.000000,0.500000,0.831295,0.000000,0.000000,0.000000,90.333333,0.000000,18.333333
Random Forest Classifier,0.290992,0.574404,0.748556,0.310916,0.283333,5.666667,75.666667,14.666667,12.666667
Gradient Boosting Classifier,0.306978,0.576768,0.693397,0.400585,0.251190,7.333333,68.000000,22.333333,11.000000
Ada Boost classifier,0.291243,0.558274,0.629148,0.452242,0.222510,8.333333,60.000000,30.333333,10.000000
Gaussian Naieve Bayes,0.274618,0.484982,0.230066,0.870370,0.163084,16.000000,9.000000,81.333333,2.333333
Logistic Regression,0.296357,0.560654,0.607402,0.490253,0.212440,9.000000,57.000000,33.333333,9.333333
Quadratic Discriminant Analysis,0.031746,0.497722,0.815947,0.017544,0.166667,0.333333,88.333333,2.000000,18.000000


,f1,roc_auc,accuracy,recall,precision,true_pos,true_neg,false_pos,false_neg
K Nearest Neighbours,0.167537,0.511745,0.754729,0.145224,0.199252,2.666667,79.333333,11.000000,15.666667
Support Vector Machines,0.288483,0.556012,0.634896,0.436647,0.216667,8.000000,61.000000,29.333333,10.333333
Gaussian Process,0.000000,0.500000,0.831295,0.000000,0.000000,0.000000,90.333333,0.000000,18.333333
Random Forest Classifier,0.279866,0.564617,0.745441,0.291423,0.273611,5.333333,75.666667,14.666667,13.000000
Gradient Boosting Classifier,0.289312,0.567894,0.702458,0.364522,0.242435,6.666667,69.666667,20.666667,11.666667
Ada Boost classifier,0.272445,0.552572,0.690140,0.345029,0.225791,6.333333,68.666667,21.666667,12.000000
Gaussian Naieve Bayes,0.255666,0.552819,0.693199,0.342105,0.209528,6.333333,69.000000,21.333333,12.000000
Logistic Regression,0.288277,0.502827,0.196228,0.964912,0.169629,17.666667,3.666667,86.666667,0.666667
Quadratic Discriminant Analysis,0.068376,0.463024,0.671027,0.148148,0.044444,2.666667,70.333333,20.000000,15.666667


,f1,roc_auc,accuracy,recall,precision,true_pos,true_neg,false_pos,false_neg
K Nearest Neighbours,0.275144,0.532978,0.549185,0.508772,0.188578,9.333333,50.333333,40.000000,9.000000
Support Vector Machines,0.233439,0.523876,0.677823,0.291423,0.196047,5.333333,68.333333,22.000000,13.000000
Gaussian Process,0.150014,0.493775,0.723893,0.146199,0.155806,2.666667,76.000000,14.333333,15.666667
Random Forest Classifier,0.260943,0.546139,0.690225,0.328460,0.217094,6.000000,69.000000,21.333333,12.333333
Gradient Boosting Classifier,0.270472,0.550821,0.687139,0.345029,0.223793,6.333333,68.333333,22.000000,12.000000
Ada Boost classifier,0.270225,0.523861,0.571101,0.453216,0.206281,8.333333,53.666667,36.666667,10.000000
Gaussian Naieve Bayes,0.268693,0.506829,0.432439,0.618908,0.171726,11.333333,35.666667,54.666667,7.000000
Logistic Regression,0.298059,0.530238,0.291284,0.890838,0.179028,16.333333,15.333333,75.000000,2.000000
Quadratic Discriminant Analysis,0.252112,0.517671,0.582824,0.419103,0.180556,7.666667,55.666667,34.666667,10.666667


,f1,roc_auc,accuracy,recall,precision,true_pos,true_neg,false_pos,false_neg
K Nearest Neighbours,0.235951,0.501573,0.579907,0.383041,0.170900,7.000000,56.000000,34.333333,11.333333
Support Vector Machines,0.262927,0.517571,0.546070,0.473684,0.185417,8.666667,50.666667,39.666667,9.666667
Gaussian Process,0.164396,0.493816,0.711802,0.164717,0.165079,3.000000,74.333333,16.000000,15.333333
Random Forest Classifier,0.249030,0.539300,0.690169,0.310916,0.212469,5.666667,69.333333,21.000000,12.666667
Gradient Boosting Classifier,0.246282,0.533278,0.668734,0.328460,0.199603,6.000000,66.666667,23.666667,12.333333
Ada Boost classifier,0.247113,0.478584,0.423378,0.562378,0.158421,10.333333,35.666667,54.666667,8.000000
Gaussian Naieve Bayes,0.231782,0.486864,0.533299,0.417154,0.164840,7.666667,50.333333,40.000000,10.666667
Logistic Regression,0.293175,0.511598,0.199286,0.982456,0.172457,18.000000,3.666667,86.666667,0.333333
Quadratic Discriminant Analysis,0.236361,0.508722,0.546466,0.452242,0.168651,8.333333,51.000000,39.333333,10.000000


It is clear that none of the methods have really "cracked this nut" so to
speak. Most models identify under 50% of the patients that need hospital
treatment and are right far less than 25% of the time when they do predict
the need for hospital treatment.

Almost all of the best results appear to come from data that is one-hot
encoded and when the target is weighted (rather than synthesising new
examples). The best performing models appear to be the tree based methods
 (Random Forest Classifier, Gradient Bossting Classifier, and Ada Boost
 Classifier), classic Logistic Regression (a version of linear regression
 optimised for classification tasks) and Support Vector Machines (trained
 using target encoded data).

We can test how well these models perform with new data by evaluating their predictions
on the holdout test set: a dataset that has not been used at any point
during training and thus is a good indicator of a
model's ability to generalise. I'll only evaluate the models identified
above, as indiscriminately evaluating every possible model against the holdout
test set risks biasing our selection - the more models we test the more
likely a result is to have occurred by chance rather than accurate modelling:

In [19]:
X_train_ohe, X_test_ohe = encode_and_scale(X_train, y_train, X_test,
                                           cat_encoder="one_hot")
X_train_ohe_scaled, X_test_ohe_scaled = encode_and_scale(X_train, y_train,
                                                         X_test,
                                                         cat_encoder="one_hot",
                                                         scaled=True)
X_train_target_scaled, X_test_target_scaled = encode_and_scale(X_train, y_train,
                                                               X_test,
                                                               cat_encoder="one_hot",
                                                               scaled=True)

best_performing_models = [
    ("one_hot_balanced", 'Random Forest Classifier'),
    ("one_hot_balanced", 'Gradient Boosting Classifier'),
    ("one_hot_balanced", 'Ada Boost classifier'),
    ("one_hot_balanced",  'Logistic Regression'),
    ("target_balanced", "Support Vector Machines")
]
test_scores = {}
for data_prep_type, model in best_performing_models:
    model_args = techniques_dict[model](balanced=True)
    clf = model_args["clf"]
    params = best_params[data_prep_type][model]
    if "weight_y" in model_args.keys():
        weight_y = model_args["weight_y"]
    else:
        weight_y = False
    if model_args["scaled"]:
        if weight_y:
            # calculate array of weights for y labels
            pos_weight, neg_weight = compute_class_weight(
                class_weight="balanced",
                classes=[1,0],
                y=y_train)
            y_weights = y_train.apply(lambda y: pos_weight if y else neg_weight)
            # train model using parameters, weights and cv loop data
            clf = (clf
                      .set_params(**params)
                      .fit(X_train_ohe_scaled,
                           y_train,
                           sample_weight=y_weights))
        else:
            # train model using parameters and cv loop data
            clf = (clf
                      .set_params(**params)
                      .fit(X_train_ohe_scaled, y_train))
    else:
        if weight_y:
            # calculate array of weights for y labels
            pos_weight, neg_weight = compute_class_weight(
                class_weight="balanced",
                classes=[1,0],
                y=y_train)
            y_weights = y_train.apply(lambda y: pos_weight if y else neg_weight)
            # train model using parameters, weights and cv loop data
            clf = (clf
                      .set_params(**params)
                      .fit(X_train_ohe,
                           y_train,
                           sample_weight=y_weights))
        else:
            # train model using parameters and cv loop data
            clf = (clf
                      .set_params(**params)
                      .fit(X_train_ohe, y_train))
    if model_args["scaled"]:
        scores = score_classifier(clf, X_test_ohe_scaled, y_test)
    else:
        scores = score_classifier(clf, X_test_ohe, y_test)
    test_scores[model] = scores

In [20]:
pd.DataFrame(test_scores).T

,f1,roc_auc,accuracy,recall,precision,true_pos,true_neg,false_pos,false_neg
Random Forest Classifier,0.240000,0.547678,0.763975,0.222222,0.260870,6.0,117.0,17.0,21.0
Gradient Boosting Classifier,0.233333,0.532615,0.714286,0.259259,0.212121,7.0,108.0,26.0,20.0
Ada Boost classifier,0.276423,0.520039,0.447205,0.629630,0.177083,17.0,55.0,79.0,10.0
Logistic Regression,0.263158,0.539663,0.652174,0.370370,0.204082,10.0,95.0,39.0,17.0
Support Vector Machines,0.333333,0.598673,0.627329,0.555556,0.238095,15.0,86.0,48.0,12.0


The test scores seem to agree reasonably with the validation scores, so we
seem to have reasonable estimates of the performance of these models (which is
 a relief given the amount of time it took to write the custom cv / grid
 search loop!)

## Closing Comments:

As I've mentioned a few times, I'm underway with a notebook to follow this
that includes a more in-depth analysis of the above models, with:
  * discussion of the model alogrithms
  * an explanations of the features the models are using to make predictions
  * analysis of individual examples the models are getting right / wrong

As things stand, none of these models are exhibiting a level of accuracy that
would leave us confident they could provide much in the way of useful
inference, or be taken forward into production. Even with better data, we
have a very long way to go to see really high levels of accuracy (the likes
of 80-90% precision and recall).

 It's because of this that I'm keen on moving away from the "rigid" single
 estimate of probability approach, to a model that can say more about the
 uncertainty of a given estimate. That way, even if the model is only right
 one third of the time, if it's confident about that third and unconfident
 the rest of the time then we can provide some useful inference when taking
 ACE referrals.

In [21]:
interesting_models = [
    ("one_hot_balanced", 'Gradient Boosting Classifier'),
    ("one_hot_balanced",  'Logistic Regression'),
    ("target_balanced", "Support Vector Machines")
]

for data_prep_type, model in interesting_models:
    print('=' * 50)
    print(model)
    print('=' * 50)
    print("Params:")
    for param, value in best_params[data_prep_type][model].items():
        print(f"{param}: {value}")


Gradient Boosting Classifier
Params:
learning_rate: 0.02
max_depth: 4
max_features: 0.2
min_samples_split: 30
n_estimators: 130
Logistic Regression
Params:
C: 0.03359818286283781
class_weight: balanced
penalty: l2
solver: liblinear
Support Vector Machines
Params:
C: 10000.0
class_weight: balanced
gamma: 0.0001
kernel: linear
